In [1]:
pip install minio pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.1/95.1 kB 1.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 8.8 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [31]:
import pandas as pd
from minio import Minio
from io import BytesIO

# Conecta no MinIO (objeto cliente)
client = Minio(
    "minio:9000",             # Host e porta do MinIO (docker ou local)
    access_key="minioadmin",  # Usuário padrão do MinIO local
    secret_key="minioadmin",  # Senha padrão do MinIO local
    secure=False              # Não usar HTTPS porque é ambiente local
)

# Nome do bucket e do arquivo CSV
bucket_name = "vendas"
object_name = "vendas.csv"

# Faz download do arquivo como um stream
response = client.get_object(bucket_name, object_name)

# Lê o conteúdo do CSV como um DataFrame do Pandas
df = pd.read_csv(BytesIO(response.read()))


In [32]:
# Exibe as primeiras 5 linhas do DataFrame
df.head()


,id_venda,data_venda,produto,quantidade,preco_unitario,valor_total,cliente
0,1,2024-11-30,Monitor,6,480.84,2885.04,Cliente B
1,2,2024-05-23,Impressora,5,4087.96,20439.80,Cliente D
2,3,2024-07-30,Teclado,6,1898.87,11393.22,Cliente C
3,4,2024-09-09,Impressora,6,2656.17,15937.02,Cliente C
4,5,2024-05-08,Impressora,7,605.91,4241.37,Cliente C


In [33]:
# Mostra as informações gerais do DataFrame (colunas, tipos de dados, valores nulos)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id_venda        1000 non-null   int64  
 1   data_venda      1000 non-null   object 
 2   produto         1000 non-null   object 
 3   quantidade      1000 non-null   int64  
 4   preco_unitario  1000 non-null   float64
 5   valor_total     1000 non-null   float64
 6   cliente         1000 non-null   object 
dtypes: float64(2), int64(2), object(3)
memory usage: 54.8+ KB


In [ ]:
# Exibe estatísticas descritivas das colunas numéricas
df.describe()


In [34]:
# Filtra o DataFrame para mostrar apenas as linhas onde o produto é 'Impressora'
df[df["produto"] == "Impressora"]


,id_venda,data_venda,produto,quantidade,preco_unitario,valor_total,cliente
1,2,2024-05-23,Impressora,5,4087.96,20439.80,Cliente D
3,4,2024-09-09,Impressora,6,2656.17,15937.02,Cliente C
4,5,2024-05-08,Impressora,7,605.91,4241.37,Cliente C
9,10,2024-10-09,Impressora,7,3979.03,27853.21,Cliente E
13,14,2024-02-04,Impressora,5,4534.16,22670.80,Cliente B
...,...,...,...,...,...,...,...
977,978,2024-01-29,Impressora,6,3240.13,19440.78,Cliente C
987,988,2024-08-26,Impressora,2,61.47,122.94,Cliente B
994,995,2024-08-02,Impressora,8,4118.54,32948.32,Cliente A
996,997,2024-10-19,Impressora,8,4427.90,35423.20,Cliente A


In [35]:
# Agrupa os dados por cliente e soma o valor_total
df.groupby("cliente")["valor_total"].sum().reset_index()


,cliente,valor_total
0,Cliente A,2547500.44
1,Cliente B,2400801.15
2,Cliente C,2604511.76
3,Cliente D,2264802.09
4,Cliente E,2623477.23


In [36]:
# Cria uma nova coluna chamada 'preco_medio' (valor_total dividido pela quantidade)
df["preco_medio"] = df["valor_total"] / df["quantidade"]

# Exibe novamente para verificar a nova coluna
df.head()


,id_venda,data_venda,produto,quantidade,preco_unitario,valor_total,cliente,preco_medio
0,1,2024-11-30,Monitor,6,480.84,2885.04,Cliente B,480.84
1,2,2024-05-23,Impressora,5,4087.96,20439.80,Cliente D,4087.96
2,3,2024-07-30,Teclado,6,1898.87,11393.22,Cliente C,1898.87
3,4,2024-09-09,Impressora,6,2656.17,15937.02,Cliente C,2656.17
4,5,2024-05-08,Impressora,7,605.91,4241.37,Cliente C,605.91


In [37]:
# Ordena o DataFrame pela coluna 'valor_total' de forma decrescente
df.sort_values(by="valor_total", ascending=False)


,id_venda,data_venda,produto,quantidade,preco_unitario,valor_total,cliente,preco_medio
813,814,2024-12-06,Impressora,9,4956.75,44610.75,Cliente B,4956.75
691,692,2024-03-15,Monitor,9,4945.69,44511.21,Cliente C,4945.69
731,732,2024-08-30,Monitor,9,4920.01,44280.09,Cliente C,4920.01
727,728,2024-01-02,Teclado,9,4855.29,43697.61,Cliente B,4855.29
489,490,2024-09-30,Notebook,9,4838.41,43545.69,Cliente A,4838.41
...,...,...,...,...,...,...,...,...
444,445,2024-10-26,Monitor,2,84.79,169.58,Cliente B,84.79
735,736,2024-06-07,Monitor,2,64.46,128.92,Cliente B,64.46
987,988,2024-08-26,Impressora,2,61.47,122.94,Cliente B,61.47
374,375,2024-07-07,Mouse,1,59.31,59.31,Cliente E,59.31


In [38]:
# Remove a coluna 'preco_medio' do DataFrame
df.drop(columns=["preco_medio"], inplace=True)

# Exibe as colunas restantes
df.head()


,id_venda,data_venda,produto,quantidade,preco_unitario,valor_total,cliente
0,1,2024-11-30,Monitor,6,480.84,2885.04,Cliente B
1,2,2024-05-23,Impressora,5,4087.96,20439.80,Cliente D
2,3,2024-07-30,Teclado,6,1898.87,11393.22,Cliente C
3,4,2024-09-09,Impressora,6,2656.17,15937.02,Cliente C
4,5,2024-05-08,Impressora,7,605.91,4241.37,Cliente C


In [39]:
# Renomeia as colunas: 'id_venda' para 'ID' e 'produto' para 'Produto'
df.rename(columns={"id_venda": "ID", "produto": "Produto"}, inplace=True)

# Visualiza o resultado
df.head()


,ID,data_venda,Produto,quantidade,preco_unitario,valor_total,cliente
0,1,2024-11-30,Monitor,6,480.84,2885.04,Cliente B
1,2,2024-05-23,Impressora,5,4087.96,20439.80,Cliente D
2,3,2024-07-30,Teclado,6,1898.87,11393.22,Cliente C
3,4,2024-09-09,Impressora,6,2656.17,15937.02,Cliente C
4,5,2024-05-08,Impressora,7,605.91,4241.37,Cliente C


In [40]:
# Agrupa os dados por 'cliente' e 'Produto', somando o 'valor_total'
df.groupby(["cliente", "Produto"])["valor_total"].sum().reset_index()


,cliente,Produto,valor_total
0,Cliente A,Fone de Ouvido,390906.47
1,Cliente A,Impressora,367820.94
2,Cliente A,Monitor,440287.75
3,Cliente A,Mouse,416468.32
4,Cliente A,Notebook,549978.34
5,Cliente A,Teclado,382038.62
6,Cliente B,Fone de Ouvido,408128.64
7,Cliente B,Impressora,488764.75
8,Cliente B,Monitor,546419.86
9,Cliente B,Mouse,297681.59


In [41]:
# Reseta o índice para voltar ao padrão numérico sequencial
df.reset_index(drop=True, inplace=True)


In [43]:
# Remove linhas duplicadas do DataFrame
df.drop_duplicates(inplace=True)


In [44]:
# Preenche os valores nulos da coluna 'preco_unitario' com zero
df["preco_unitario"].fillna(0, inplace=True)


In [45]:
# Remove linhas que tenham valores nulos nas colunas especificadas
df.dropna(subset=["cliente", "Produto"], inplace=True)


In [47]:
# Salva o DataFrame final em um arquivo CSV local
df.to_csv("vendas_tratado.csv", index=False)
